In [13]:
# Importar bibliotecas necessárias
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import pandas as pd
import numpy as np
import chromadb
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [14]:
# Carregar dados reais de carros a partir do arquivo CSV
car_df = pd.read_csv("car_data.csv")  # Assegure-se de salvar o CSV com o nome "car_data_pt.csv"

# Debug: Mostrar os dados dos carros carregados
print("Dados dos carros carregados:")
print(car_df.head())

# Preparar descrições dos carros
car_descriptions = car_df.apply(
    lambda x: f"{x['ano']} {x['fabricante']} {x['modelo']} com {x['potencia']} HP, {x['cavalos_rodas']} cilindros, "
                  f"rodas motrizes {x['rodas_motrizes']}, consumo na estrada de {x['consumo_estrada']} MPG, preço R${x['preco']}",
            axis=1
    ).tolist()


Dados dos carros carregados:
  fabricante      modelo   ano  potencia  cavalos_rodas rodas_motrizes  \
0  Chevrolet      Camaro  2020       455              8            RWD   
1       Ford     Mustang  2020       450              8            RWD   
2      Dodge  Challenger  2020       485              8            RWD   
3        BMW          M3  2020       473              6            RWD   
4       Audi         RS5  2020       444              6            AWD   

   consumo_estrada  preco  
0               27  30000  
1               25  35000  
2               23  32000  
3               26  70000  
4               27  75000  


In [ ]:
print("Descrições dos carros:")
for desc in car_descriptions:
    print(desc)

In [8]:
# Normalizar vetores para similaridade de cosseno
def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

In [ ]:
 # Preparar embeddings
bi_encoder_model = SentenceTransformer('all-MiniLM-L6-v2')
car_embeddings = bi_encoder_model.encode(car_descriptions)

client = chromadb.Client()
client.delete_collection(name="car_collection")
        
collection = client.create_collection(name="car_collection", metadata={"hnsw:space": "cosine"})

# Adicionar embeddings ao ChromaDB
for i, embedding in enumerate(car_embeddings):
    collection.add(ids=[str(i)], embeddings=[embedding.tolist()])

# Consulta do cliente
client_query = "carros esportivos com alta performance e baixo custo"

# Transformar a consulta em embedding
client_query_embedding = bi_encoder_model.encode(client_query)

# Debug: Mostrar embedding da consulta
print("Embedding da consulta:")
print(client_query_embedding)

In [26]:
# Busca semântica para recuperação inicial usando ChromaDB
results = collection.query(query_embeddings=[client_query_embedding.tolist()], n_results=5)
initial_results_indices = [int(id) for id in results['ids'][0]]
initial_scores = results['distances'][0]

initial_results = car_df.iloc[initial_results_indices].reset_index(drop=True)
initial_results_descriptions = [car_descriptions[idx] for idx in initial_results_indices]
initial_results_embeddings = car_embeddings[initial_results_indices]


In [27]:
# Debug: Mostrar resultados iniciais da busca
print("Resultados iniciais da busca (descrições e pontuações):")
for desc, score in zip(initial_results_descriptions, initial_scores):
    print(f"{desc} - Pontuação: {score}")


Resultados iniciais da busca (descrições e pontuações):
2020 Ferrari Portofino com 591 HP, 8 cilindros, rodas motrizes RWD, consumo na estrada de 22 MPG, preço R$215000 - Pontuação: 0.6434164047241211
2020 Renault Sandero com 150 HP, 4 cilindros, rodas motrizes FWD, consumo na estrada de 28 MPG, preço R$60000 - Pontuação: 0.6881915330886841
2020 Porsche 911 Carrera com 379 HP, 6 cilindros, rodas motrizes RWD, consumo na estrada de 30 MPG, preço R$97000 - Pontuação: 0.6919667720794678
2020 Fiat Palio com 88 HP, 4 cilindros, rodas motrizes FWD, consumo na estrada de 30 MPG, preço R$45000 - Pontuação: 0.7019725441932678
2020 Audi RS5 com 444 HP, 6 cilindros, rodas motrizes AWD, consumo na estrada de 27 MPG, preço R$75000 - Pontuação: 0.7202595472335815


In [28]:
# Criar tabela de resultados iniciais
initial_results['score'] = initial_scores
print("Tabela de resultados iniciais:")
print(initial_results[['fabricante', 'modelo', 'ano', 'potencia', 'cavalos_rodas', 'preco', 'score']])


Tabela de resultados iniciais:
  fabricante       modelo   ano  potencia  cavalos_rodas   preco     score
0    Ferrari    Portofino  2020       591              8  215000  0.643416
1    Renault      Sandero  2020       150              4   60000  0.688192
2    Porsche  911 Carrera  2020       379              6   97000  0.691967
3       Fiat        Palio  2020        88              4   45000  0.701973
4       Audi          RS5  2020       444              6   75000  0.720260


In [29]:
 # Preparar pares para o Cross-Encoder
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6')
cross_encoder_inputs = [(client_query, desc) for desc in initial_results_descriptions]
cross_encoder_scores = cross_encoder_model.predict(cross_encoder_inputs)

    # Debug: Mostrar pontuações do Cross-Encoder
print("Pontuações do Cross-Encoder:")
print(cross_encoder_scores)

Pontuações do Cross-Encoder:
[0.0001693  0.00016075 0.00143747 0.00016192 0.00016317]


In [30]:
# Re-Ranking com Cross-Encoder
initial_results['relevance'] = cross_encoder_scores
reordered_results = initial_results.sort_values(by='relevance', ascending=False).reset_index(drop=True)

In [32]:
# Debug: Mostrar descrições e relevâncias após o re-ranking
print("Descrições e relevâncias após o re-ranking:")
print(reordered_results[['fabricante', 'modelo', 'relevance', 'preco']])


Descrições e relevâncias após o re-ranking:
  fabricante       modelo  relevance   preco
0    Porsche  911 Carrera   0.001437   97000
1    Ferrari    Portofino   0.000169  215000
2       Audi          RS5   0.000163   75000
3       Fiat        Palio   0.000162   45000
4    Renault      Sandero   0.000161   60000


In [33]:
# Criar tabela de resultados após o re-ranking
print("Tabela de resultados após o re-ranking:")
print(reordered_results[['fabricante', 'modelo', 'ano', 'potencia', 'preco', 'relevance']])


Tabela de resultados após o re-ranking:
  fabricante       modelo   ano  potencia   preco  relevance
0    Porsche  911 Carrera  2020       379   97000   0.001437
1    Ferrari    Portofino  2020       591  215000   0.000169
2       Audi          RS5  2020       444   75000   0.000163
3       Fiat        Palio  2020        88   45000   0.000162
4    Renault      Sandero  2020       150   60000   0.000161


In [34]:
# Debug: Mostrar os índices reordenados
reordered_indices = [car_df[car_df['modelo'] == model].index[0] for model in reordered_results['modelo']]
print("Índices reordenados:")
print(reordered_indices)

Índices reordenados:
[6, 14, 4, 21, 25]


In [40]:
# Definir a função para plotar embeddings
def plot_embeddings(embeddings, car_data, query_embedding, query_label, title):
    # Reduzir dimensionalidade para 2D com PCA
    pca = PCA(n_components=2)
    reduced_embeddings = pca.fit_transform(embeddings)
    
    # Reduzir a dimensionalidade do embedding da consulta
    query_reduced = pca.transform(query_embedding.reshape(1, -1))

    # Plotar os embeddings dos carros
    plt.figure(figsize=(12, 12))
    plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c='blue', label='Carros')
    
    # Adicionar anotações aos pontos
    for i, row in car_data.iterrows():
        plt.annotate(f"{row['modelo']}\n{row['potencia']} HP, R${row['preco']}, {row['relevance']}", 
                     (reduced_embeddings[i, 0], reduced_embeddings[i, 1]))

    # Plotar o embedding da consulta
    plt.scatter(query_reduced[:, 0], query_reduced[:, 1], c='red', label='Consulta', marker='x')
    plt.annotate(query_label, (query_reduced[0, 0], query_reduced[0, 1]), color='red')

    # Configurar o título e a legenda
    plt.title(title)
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
reordered_embeddings = car_embeddings[reordered_indices]

# Visualização da busca semântica em geral
plot_embeddings(np.vstack([car_embeddings, client_query_embedding]), car_df, client_query_embedding, client_query, 'Todos os Carros e Consulta')


In [ ]:
# visualização da busca semÂntica antes da reclassificação
plot_embeddings(initial_results_embeddings, initial_results, client_query_embedding, client_query, 'Antes do Re-Ranking')

In [ ]:
# visualização da busca semÂntica após a reclassificação

plot_embeddings(reordered_embeddings, reordered_results, client_query_embedding, client_query, 'Após o Re-Ranking')